In [1]:
# Import necessary packages
import xarray as xr
import matplotlib.pyplot as plt
import intake
import s3fs

In [2]:
# Open the CMIP6 data catalog, store as a variable
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')

In [5]:
# Search through catalog, find all historical simulations
# ("activity_id=CMIP", "experiment_id=historical")
res = catalog.search(activity_id="CMIP", experiment_id="historical", member_id="r1i1p1f1")

# Convert to a data frame
res_df = res.df

# Get unique model names in the set of search results
models = res_df.source_id.unique()

# Print list of model names
print(models)

['GFDL-CM4' 'GFDL-ESM4' 'IPSL-CM6A-LR' 'GISS-E2-1-G' 'BCC-CSM2-MR'
 'BCC-ESM1' 'MIROC6' 'AWI-CM-1-1-MR' 'MRI-ESM2-0' 'CESM2-WACCM' 'CanESM5'
 'CESM2' 'SAM0-UNICON' 'GISS-E2-1-H' 'INM-CM4-8' 'EC-Earth3-Veg'
 'INM-CM5-0' 'MPI-ESM-1-2-HAM' 'NESM3' 'CAMS-CSM1-0' 'MPI-ESM1-2-LR'
 'MPI-ESM1-2-HR' 'EC-Earth3' 'MCM-UA-1-0' 'NorESM2-LM' 'GISS-E2-1-G-CC'
 'FGOALS-g3' 'FGOALS-f3-L' 'E3SM-1-0' 'KACE-1-0-G' 'NorCPM1' 'KIOST-ESM'
 'NorESM2-MM' 'ACCESS-CM2' 'FIO-ESM-2-0' 'ACCESS-ESM1-5' 'CESM2-FV2'
 'CESM2-WACCM-FV2' 'GISS-E2-2-H' 'E3SM-1-1' 'IITM-ESM' 'CIESM'
 'E3SM-1-1-ECA' 'AWI-ESM-1-1-LR' 'EC-Earth3-Veg-LR' 'TaiESM1' 'CAS-ESM2-0'
 'CMCC-CM2-SR5' 'EC-Earth3-AerChem' 'IPSL-CM5A2-INCA' 'CMCC-CM2-HR4'
 'EC-Earth3-CC' 'CMCC-ESM2' 'ICON-ESM-LR' 'IPSL-CM6A-LR-INCA']


In [7]:
catalog

,unique
activity_id,18
institution_id,36
source_id,88
experiment_id,170
member_id,657
table_id,37
variable_id,709
grid_label,10
zstore,522217
dcpp_init_year,60
